In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [5]:
df_old = pd.read_csv('../Data/Raw/rcrwireless.csv')
df_old['Category'] = df_old['Category'].apply(lambda x : eval(x))
print(set(df_old['Category'].sum()))
print(df_old.columns)
print(len(df_old))

In [49]:
path = '../Data/Preprocessed/rcrwireless2.csv'

df_origin = pd.read_csv(path)
df_origin['Date'] = df_origin['Date'].apply(lambda x : pd.to_datetime(x.split('-')[0]))
df_origin[['Keywords', 'Category', 'topicsRatio']] = df_origin[['Keywords', 'Category', 'topicsRatio']].applymap(eval)
df_origin['year'] = df_origin['Date'].dt.year
df_origin['month'] = df_origin['Date'].dt.month
df_origin.drop(columns = ['Title__stop', 'PreprocessedTitle', 'SummarizedText', 'SummarizedText__stop', 'PreprossedSummarizedText'], inplace = True)
df_origin.set_index('Date', inplace = True)

In [78]:
print(len(df_origin['2010':]))

3852


In [77]:
# 주요 키워드 별 기사 수(중복 분리)
result_df = pd.DataFrame()

for i in range(1, 5):
    condition = df_origin['Category'].apply(lambda x : len(x) == i)
    tmp = df_origin[condition]
    tmp = tmp['2010':]
    tmp.loc[:, 'Category'] = tmp.loc[:, 'Category'].apply(lambda x : str(sorted(x)))
    tmp = tmp.groupby(['Category'])['Href'].nunique().reset_index()

    result_df = pd.concat([result_df, tmp], axis = 0, ignore_index = True)  

result_df

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_25064\646235920.py:6: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  tmp = tmp['2010':]


,Category,Href
0,['6E'],361
1,['6G'],624
2,['Automotive'],1479
3,['MIMO'],1089
4,"['6E', '6G']",25
5,"['6E', 'Automotive']",18
6,"['6E', 'MIMO']",22
7,"['6G', 'Automotive']",61
8,"['6G', 'MIMO']",94
9,"['Automotive', 'MIMO']",61


In [87]:
#주요 키워드 및 연도별 기사 수(중복 포함)
category_list = ['6G', '6E', 'Automotive', 'MIMO']

df = df_origin.copy()
df = df['2010' :]
result = pd.DataFrame()
for category in category_list:
    condition = df['Category'].apply(lambda x : category in x)
    tmp = df[condition]
    tmp = tmp.groupby(['year'])['Href'].nunique().reset_index()
    tmp['Category'] = category
    result = pd.concat([result, tmp], axis = 0)

result = result.pivot_table(index = ['Category'], columns = ['year'], values = ['Href']).fillna(0).T.reset_index()
result = result.drop(columns = ['level_0'])
result

Category,year,6E,6G,Automotive,MIMO
0,2010,8,14,2,4
1,2011,35,18,16,11
2,2012,11,20,26,9
3,2013,26,32,31,32
4,2014,24,9,60,55
5,2015,21,20,108,63
6,2016,25,19,190,126
7,2017,41,20,170,180
8,2018,14,42,217,183
9,2019,16,76,209,137


In [88]:
result.sum(axis = 0)

Category
year          28231
6E              435
6G              820
Automotive     1631
MIMO           1283
dtype: int64

In [89]:
# 주요 키워드 별 기사 수(중복 분리)
result_df = pd.DataFrame()

for i in range(1, 5):
    condition = df_origin['Category'].apply(lambda x : len(x) == i)
    tmp = df_origin[condition]
    tmp = tmp['2010':]
    tmp.loc[:, 'Category'] = tmp.loc[:, 'Category'].apply(lambda x : str(sorted(x)))
    tmp = tmp.groupby(['Category']).resample('Y')['Href'].nunique().reset_index()

    result_df = pd.concat([result_df, tmp], axis = 0, ignore_index = True)  

result_df

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_25064\2324885350.py:7: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  tmp = tmp['2010':]


,Category,Date,Href,0
0,['6E'],2010-12-31,7.0,NaN
1,['6E'],2011-12-31,35.0,NaN
2,['6E'],2012-12-31,11.0,NaN
3,['6E'],2013-12-31,24.0,NaN
4,['6E'],2014-12-31,24.0,NaN
...,...,...,...,...
130,"['6G', 'Automotive', 'MIMO']",2019-12-31,2.0,NaN
131,"['6G', 'Automotive', 'MIMO']",2020-12-31,0.0,NaN
132,"['6G', 'Automotive', 'MIMO']",2021-12-31,2.0,NaN
133,"['6G', 'Automotive', 'MIMO']",2022-12-31,3.0,NaN


In [114]:
df = df_origin.copy()
result_df = pd.DataFrame()

for i in range(2, 5):
    condition = df['Category'].apply(lambda x : len(x) == i)
    tmp = df[condition]
    tmp = tmp['2010':]
    tmp.loc[:, 'Category'] = tmp.loc[:, 'Category'].apply(lambda x : str(sorted(x)))
    tmp = tmp.groupby(['Category', 'year'])['Href'].nunique().reset_index().pivot_table(index = 'Category', columns = ['year'], values = ['Href']).fillna(0)

    result_df = pd.concat([result_df, tmp], axis = 0)

result_df.fillna(0).T

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_25064\2763611842.py:7: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  tmp = tmp['2010':]


Category   ['6E', '6G']  ['6E', 'Automotive']  ['6E', 'MIMO']  \
     year                                                       
Href 2010           0.0                   0.0             1.0   
     2012           0.0                   0.0             0.0   
     2013           1.0                   0.0             1.0   
     2014           0.0                   0.0             0.0   
     2015           0.0                   0.0             0.0   
     2016           0.0                   2.0             2.0   
     2017           0.0                   0.0             3.0   
     2018           0.0                   1.0             1.0   
     2019           1.0                   2.0             1.0   
     2020          10.0                   0.0             4.0   
     2021           2.0                   5.0             4.0   
     2022           8.0                   5.0             3.0   
     2023           3.0                   3.0             2.0   

Category   ['6G', 'Automotive']  ['6G', 'MIMO']  ['Automotive', 'MIMO']  \
     year                                                                 
Href 2010                   0.0             1.0                     0.0   
     2012                   1.0             0.0                     0.0   
     2013                   0.0             1.0                     0.0   
     2014                   0.0             0.0                     3.0   
     2015                   1.0             1.0                     4.0   
     2016                   1.0             3.0                     7.0   
     2017                   1.0             3.0                     9.0   
     2018                   4.0            11.0                    13.0   
     2019                   4.0            14.0                    10.0   
     2020                   5.0            13.0                     3.0   
     2021                  10.0             9.0                     5.0   
     2022                  20.0            24.0                     7.0   
     2023                  14.0            14.0                     0.0   

Category   ['6E', '6G', 'Automotive']  ['6E', '6G', 'MIMO']  \
     year                                                     
Href 2010                         0.0                   0.0   
     2012                         0.0                   0.0   
     2013                         0.0                   0.0   
     2014                         0.0                   0.0   
     2015                         0.0                   0.0   
     2016                         0.0                   0.0   
     2017                         0.0                   0.0   
     2018                         0.0                   0.0   
     2019                         0.0                   1.0   
     2020                         0.0                   3.0   
     2021                         0.0                   1.0   
     2022                         1.0                   1.0   
     2023                         0.0                   0.0   

Category   ['6E', 'Automotive', 'MIMO']  ['6G', 'Automotive', 'MIMO']  
     year                                                              
Href 2010                           0.0                           0.0  
     2012                           0.0                           0.0  
     2013                           0.0                           0.0  
     2014                           0.0                           0.0  
     2015                           0.0                           0.0  
     2016                           0.0                           0.0  
     2017                           0.0                           0.0  
     2018                           0.0                           0.0  
     2019                           0.0                           2.0  
     2020                           0.0                           0.0  
     2021                           0.0                           2.0  
     2022       